In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# 3.1 计算图定义计算

a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = a + b
print(result)

# 3.2 张量
# 三个属性： name, shape,type
# 3.3 会话（运行模型）执行计算


with tf.Session() as sess:
    print(sess.run(result))

Tensor("add:0", shape=(2,), dtype=float32)
[ 3.  5.]


In [5]:
# 3.4.3 神经网络参数与TensorFlow变量
# #声明w1 w2 两个变量。这里还通过seed参数设定了随机种子
# #这样可以保证每次运行得到结果是一样的
# w1 2行3列
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
# w2 3行1列
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))
# [注意]w3 1行 2列
w3 = tf.Variable(tf.truncated_normal([2], stddev=0.1))
# b2 一行10列
b2 = tf.Variable(tf.constant(0.1, shape=[10]))

# 暂时将输入定义一个常量。1*2 的矩阵

x = tf.constant([[0.7,0.9]])

# 计算神经网络的输出 a为1行3列
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
# page55
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # print(sess.run(y))
    # print(b2.eval())
    # plt.plot(np.transpose(w3.eval()))
    print(w3.eval())
    # plt.show()

[ 0.17478476  0.1280534 ]


In [8]:
# placeholder 
import tensorflow as tf


w1 = tf.Variable(tf.random_normal([2, 3], stddev=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1))

# 定义placeholder作为存放输入数据的地方。这里维度也不一定要定义。
# 如果维度是确定的，那么给出维度可以降低出错的概率。

x = tf.placeholder(tf.float32, shape=(3, 2), name = "input")
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(y, feed_dict={x: [[0.7, 0.9],[0.1, 0.4],[0.5, 0.8]]}))

[[-0.77436435]
 [-0.66980338]
 [-0.87685311]]


In [6]:
# 完整神经网络样例程序，解决二分类

import tensorflow as tf


# Numpy是科学计算工具包，通过Numpy工具包生成模拟数据
from numpy.random import RandomState

# 定义训练数据的batch大小
batch_size = 8

# 定义神经网络的参数
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# 在shape 的一个维度上使用Ｎone可以方便使用不同batch大小。在训练时需要把数据
# 分成比较小的batch，但是测试时，可以一次使用全部数据。当数据集比较小时这样比较
# 方便测试，但是数据集比较大时，将大量数据放入一个batch可能会导致内存溢出。

x = tf.placeholder(tf.float32,shape=[None, 2], name="x-input")
y_ = tf.placeholder(tf.float32,shape=[None, 1], name="y-input")

# 定义神经网络的传播过程

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 定义损失函数和反向传播算法

cross_entropy = -tf.reduce_mean(
    y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
)
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# 通过随机数生成一个模拟数据集 128*2
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)

# 定义规则来给出样本的标签。这里所有x1+x2 < 1 的样例都被认为是正样本（比如零件合格），
# 而其他为负样本（比如零件不合格）。和TensorFlow游乐场中的表示法不大一样的是，在这里使用
# 0 表示负样本，1 来表示正样本。大部分分类问题的神经网络都会采用 0 和 1 的表示方法。
# int(True) -> 1


# 放在列表中的生成的是每一次生成一个列表
# from numpy.random import RandomState
# rdm = RandomState(1)
# dataset_size = 4
# X = rdm.rand(dataset_size, 2)
# Y1 = [[int(x1+x2 < 1)] for (x1,x2) in X ]
# Y2 = [int(x1+x2 < 1) for (x1,x2) in X ]
# # print(type(Y1)," == ",type(Y2))
# print(Y1)
# [[0], [1], [1], [1]]
# print(Y2)
# [0, 1, 1, 1]

# 根据X生成的标签
Y = [[int(x1+x2 < 1)] for (x1,x2) in X ]

# 创建会话运行TensorFlow程序
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    # 初始化变量
    sess.run(init_op)
    # print(sess.run(w1))
    # print(sess.run(w2))  
    # 训练指定的轮数
    STEPS = 5000
    for i in range(STEPS):
        # 每次选取batch_size个样本进行训练。 重复使用
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size,dataset_size)
        
        # 通过选取的样本训练神经网络并更新参数
        sess.run(
            train_step, feed_dict={x: X[start:end], y_: Y[start:end]}
        )
        
        if i % 1000 == 0:
            total_cross_entropy = sess.run(
                cross_entropy, feed_dict={x: X, y_: Y}
            )
            print("After %d training steps,cross entropy on all data is %g" %(i, total_cross_entropy))
            print(sess.run(w1)) 
            print(sess.run(w2))

After 0 training steps,cross entropy on all data is 0.0674925
[[-0.81231821  1.4855988   0.06632921]
 [-2.44370413  0.1002484   0.59222424]]
[[-0.81231821]
 [ 1.4855988 ]
 [ 0.06632937]]


After 1000 training steps,cross entropy on all data is 0.0163385
[[-1.27549374  1.93239319  0.71818316]
 [-2.82764411  0.47066161  1.14189851]]
[[-1.21198606]
 [ 1.95956361]
 [ 0.55081969]]


After 2000 training steps,cross entropy on all data is 0.00907547
[[-1.51397622  2.1591146   1.07429051]
 [-3.01708913  0.64845419  1.46364999]]
[[-1.40506411]
 [ 2.20634365]
 [ 0.83948904]]


After 3000 training steps,cross entropy on all data is 0.00714436
[[-1.65394425  2.29218411  1.27439237]
 [-3.14156055  0.76467752  1.66820383]]
[[-1.52613485]
 [ 2.35394239]
 [ 1.01985705]]


After 4000 training steps,cross entropy on all data is 0.00578471
[[-1.79143536  2.42184758  1.46388769]
 [-3.28938985  0.90241849  1.88527477]]
[[-1.66073918]
 [ 2.50406837]
 [ 1.20711744]]
